In [2]:
# Imports
import os
import pandas as pd
import numpy as np

In [10]:
# Read in the data
directory = '../data/'
extension = '.csv'
columns = ['FTR', 'AvgH', 'AvgD', 'AvgA', 'B365H', 'B365D', 'B365A']

df = pd.DataFrame()

# Iterate through all sub-directories and files
for dirpath, dirnames, files in os.walk(directory):
    for file in files:
        if file.endswith(extension):
            # Open the file and read its contents
            contents = pd.read_csv(os.path.join(dirpath, file))
            df = df.append(contents[columns])

# Print the first 5 rows of the DataFrame
df.head()

/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_39701/3453979798.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[columns])
/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_39701/3453979798.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[columns])
/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_39701/3453979798.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(contents[columns])


,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA
0,Fulham,Arsenal,0,3,A,5.94,4.34,1.55
1,Crystal Palace,Southampton,1,0,H,3.18,3.22,2.39
2,Liverpool,Leeds,4,3,H,1.30,5.96,9.68
3,West Ham,Newcastle,0,2,A,2.15,3.48,3.42
4,West Brom,Leicester,0,3,A,3.87,3.57,1.97


In [18]:
# Agora, vamos analisar a taxa de acertos das faixas de odds
# Indo de 1.0 a 10.0, com 0.1 de intervalo, temos 99 faixas
# Vamos criar um dataframe com essas faixas e inserir a quantidade de acertos
# FTR será utilizado para dizer qual foi o resultado do jogo: H, D ou A
# AvgH, AvgD e AvgA serão utilizados para dizer qual foi a odd média do jogo

# Todas as colunas que possuem odds:
oddsColumns = ['AvgH', 'AvgD', 'AvgA']

# Arredondando as oddsColums para apenas 1 casa decimal:
for column in oddsColumns:
    df[column] = df[column].round(1)

# Criando as faixas de odds:
faixas = np.linspace(1.0, 10.0, 99).round(1)

# Colunas do dataframe que será criado:
# % Odd: 1 / odd - percentual de acerto que a odd teria que ter para ser lucrativa
# % Real: (quantidade de acertos) / (quantidade de jogos) - percentual de acerto real
# AvgHT: quantidade de jogos com odd média de vitória do time da casa igual a faixa
# AvgHR: quantidade de acertos com odd média de vitória do time da casa igual a faixa
# AvgDT: quantidade de jogos com odd média de empate igual a faixa
# AvgDR: quantidade de acertos com odd média de empate igual a faixa
# AvgAT: quantidade de jogos com odd média de vitória do time visitante igual a faixa
# AvgAR: quantidade de acertos com odd média de vitória do time visitante igual a faixa
faixasColumns = ['% Odd', '% Real', 'AvgHT', 'AvgHR', 'AvgDT', 'AvgDR', 'AvgAT', 'AvgAR']

# Criando o dataframe:
dfFaixas = pd.DataFrame(0, index=faixas, columns=faixasColumns)

# Extraindo as odds médias de cada resultado:
FTRH = df['AvgH'][df['FTR'] == 'H']
FTRD = df['AvgD'][df['FTR'] == 'D']
FTRA = df['AvgA'][df['FTR'] == 'A']

# Preenchendo o dataframe com as informações:
for faixa in faixas:
    dfFaixas.at[faixa, 'AvgHT'] = (df['AvgH'] == faixa).sum()
    dfFaixas.at[faixa, 'AvgHR'] = (FTRH == faixa).sum()
    dfFaixas.at[faixa, 'AvgDT'] = (df['AvgD'] == faixa).sum()
    dfFaixas.at[faixa, 'AvgDR'] = (FTRD == faixa).sum()
    dfFaixas.at[faixa, 'AvgAT'] = (df['AvgA'] == faixa).sum()
    dfFaixas.at[faixa, 'AvgAR'] = (FTRA == faixa).sum()
    dfFaixas.at[faixa, '% Odd'] = (1/faixa).round(2)
    dfFaixas.at[faixa, '% Real'] = ((
        dfFaixas.at[faixa, 'AvgHR'] + dfFaixas.at[faixa, 'AvgDR'] + dfFaixas.at[faixa, 'AvgAR']) / (
            dfFaixas.at[faixa, 'AvgHT'] + dfFaixas.at[faixa, 'AvgDT'] + dfFaixas.at[faixa, 'AvgAT'])).round(2)

# Visualizando as linhas em que a taxa de acerto é maior que a taxa de odd, e a quantidade total de jogos é maior que 10 * quantidade de files
# Imprimir apenas o indice e as colunas % Odd, % Real
dfFaixas[(dfFaixas['% Real'] > dfFaixas['% Odd']) & ((dfFaixas['AvgHT'] + dfFaixas['AvgDT'] + dfFaixas['AvgAT']) > 10 * len(files))][['% Odd', '% Real']]

/var/folders/7y/v7whc3tn0fl5rddh_2ql4n3m0000gn/T/ipykernel_39701/405327070.py:45: RuntimeWarning: invalid value encountered in long_scalars
  dfFaixas.at[faixa, '% Real'] = ((


,% Odd,% Real
1.5,0.67,0.69
1.8,0.56,0.61
2.3,0.43,0.48
2.4,0.42,0.43
2.8,0.36,0.38
3.2,0.31,0.32
3.3,0.30,0.32
3.5,0.29,0.33
4.1,0.24,0.35
4.3,0.23,0.26
